# OMF-Project-Collection

This notebook aggregates all the data for the FORGE site into a single Open Mining Format (OMF) project file.

In [ ]:
import sys
# This makes the gdc19 package avaialble
sys.path.append('../../')
import gdc19
gdc19.setup_bane()

In [ ]:
import vtki
import omf
import omfvtk
import pandas as pd
import numpy as np

## Surfaces

This notebook loads all the surface data and creates `omf.SurfaceElement` objects.

In [ ]:
def surf_to_omf(filename, name, description, elevation=False):
    surf = gdc19.delauney(
        gdc19.read_surface_verts(
            gdc19.get_surfaces_path(filename)))
    if elevation:
        surf = surf.elevation()
    
    tris = surf.faces.reshape(surf.n_cells, 4)[:, 1:4]
    element = omf.SurfaceElement(
        name=name,
        description=description,
        geometry=omf.SurfaceGeometry(vertices=surf.points,
                                     triangles=tris)
        )
    element.validate()
    return element

In [ ]:
description='This contains vertices of meshed/interpolated ' \
    'surfaces of the granitoid-basin fill contact used in the ' \
    'Phase 2B earth model. All data are georeferenced to UTM, ' \
    'zone 12N, NAD 83, NAVD 88.'

top_granitoid = surf_to_omf(
    'top_granitoid_vertices.csv', 
    'top_granitoid',
    description
    )
top_granitoid.validate()

In [ ]:
omfvtk.wrap(top_granitoid).plot(show_edges=False)

In [ ]:
description='Negro Mag Fault used in the Phase 2B earth model. ' \
    'All data are georeferenced to UTM, zone 12N, NAD 83, NAVD 88.'

negro_mag_fault = surf_to_omf(
    'Negro_Mag_Fault_vertices.csv', 
    'negro_mag_fault',
    description
    )
negro_mag_fault.validate()

In [ ]:
 omfvtk.wrap(negro_mag_fault).plot(show_edges=False)

In [ ]:
description='surfaces of the Opal Mound Fault used in the Phase ' \
    '2B earth model. All data are georeferenced to UTM, zone '\
    '12N, NAD 83, NAVD 88.'

opal_mound_fault = surf_to_omf(
    'Opal_Mound_Fault_vertices.csv', 
    'opal_mound_fault',
    description
    )
opal_mound_fault.validate()

In [ ]:
omfvtk.wrap(opal_mound_fault).plot(show_edges=False)

In [ ]:
# temp_175c: '175C_vertices.csv'
description='vertices of meshed/interpolated surfaces of the ' \
    'interpolated temperature isosurfaces for 175 degrees C used ' \
    'in the Phase 2B earth model. All data are georeferenced to ' \
    'UTM, zone 12N, NAD 83, NAVD 88.'

temp_175c = surf_to_omf(
    '175C_vertices.csv', 
    'temp_175c',
    description
    )
temp_175c.validate()

In [ ]:
omfvtk.wrap(temp_175c).plot(show_edges=False)

In [ ]:
# temp_225c: '225C_vertices.csv'
description='vertices of meshed/interpolated surfaces of the '\
    'interpolated temperature isosurfaces for 225 degrees C used '\
    'in the Phase 2B earth model. All data are georeferenced to ' \
    'UTM, zone 12N, NAD 83, NAVD 88.'

temp_225c = surf_to_omf(
    '225C_vertices.csv', 
    'temp_225c',
    description
    )
temp_225c.validate()

In [ ]:
omfvtk.wrap(temp_225c).plot(show_edges=False)

In [ ]:
# land_surface: 'land_surface_vertices.csv'
description='vertices of meshed/interpolated surfaces of the ' \
    'land surface (based on 10-meter DEM) used in the Phase 2B '\
    'earth model. All data are georeferenced to UTM, zone 12N, '\
    'NAD 83, NAVD 88.'

land_surface = surf_to_omf(
    'land_surface_vertices.csv', 
    'land_surface',
    description
    )
land_surface.validate()

In [ ]:
topo = omfvtk.wrap(land_surface)
topo.plot(show_edges=False)

In [ ]:
# THIS DOES NOT WORK! TEXTURES HAVE TO BE MANAGED ON THE VTK SIDE
# # NOW ADD TEXTURES TO THE LAND SURFACE

# fname = gdc19.get_gis_path('Geologic_map_on_air_photo.png')
# name = 'geo_air'
# description = 'geologic map of the Utah FORGE site with aerial '\
#     'imagery. All data are georeferenced to UTM, zone 12N, NAD '\
#     '83, NAVD 88.'
# geo_air = gdc19.load_texture_to_omf(fname, name, description)

# # topographic_map.png
# fname = gdc19.get_gis_path('topographic_map.png')
# name = 'topo_map'
# description = ' topographic map of the Utah FORGE site. '\
#     'All data are georeferenced to UTM, zone 12N, NAD 83, NAVD 88.'
# topo_map = gdc19.load_texture_to_omf(fname, name, description)

# # Geologic_map_no_quaternary.tif
# fname = gdc19.get_gis_path('Geologic_map_no_quaternary.png')
# name = 'geo_no_air'
# description = 'geologic map of the Utah FORGE site with '\
#     'quaternary faults labeled. All data are georeferenced to UTM, '\
#     'zone 12N, NAD 83, NAVD 88.'
# geo_no_air = gdc19.load_texture_to_omf(fname, name, description)

# land_surface.textures = [geo_air, topo_map, geo_no_air]
# land_surface.validate()

## GIS Data

```
conda install gdal
```

Please not that textures must be hanlded on the VTK side due to issues within OMF

```py
topo = omfvtk.wrap(land_surface)
# Use convieance method to bind all the textures to the topography
gdc19.attach_all_textures(topo)
topo.plot(show_edges=False, texture=True)
```

In [ ]:
# Load the shape files
shapes = gdc19.read_shape_file_to_omf(
    gdc19.get_shp_path('FORGE_Outline'), topo_points=topo.points
)
shapes

In [ ]:
boundary = shapes[0]
boundary.name = 'boundary'
boundary.validate()

In [ ]:
omfvtk.wrap(boundary).plot(show_edges=False)

## Temperature Data

In [ ]:
# Load the temperature data
_temp = pd.read_csv(
            gdc19.get_temperature_path('well_based_temperature.csv')
            )
_temp.head()

In [ ]:
temperature = omf.PointSetElement(
    name='temperature',
    description='cumulative record of one-dimensional temperature modeling '\
        'based off of well data. Temperature log data were exampled and '\
        'extrapolated below the bottom of a number of wells. Temperatures '\
        'are in degrees Celsius, and all location data are georeferenced to '
        'UTM, zone 12N, NAD 83, NAVD 88.',
    subtype='point',
    geometry=omf.PointSetGeometry(
        vertices=_temp[['x', 'y', 'z']].values
    ),
    data=[omf.ScalarData(
        name='temperature (C)',
        array=_temp['T'].values,
        location='vertices'
    ),]
)
temperature.validate()

In [ ]:
omfvtk.wrap(temperature).plot(show_edges=False)

## Well Data

Incomplete.

We'll use some of [`lasio`](https://lasio.readthedocs.io/en/latest/) for the file IO

In [ ]:
gdc19.list_filenames(gdc19.get_well_path())

In [ ]:
import lasio
las = lasio.read(gdc19.get_well_path(r'Acord 1-26_digitized_geophysical_well_log.las'))
las.curvesdict.keys()

In [ ]:
# plt.plot(las.index, las["GAMMA_RAY_SPLICE"])

In [ ]:
# TODO

## Drilling Data

In [ ]:
gdc19.get_drilling_path('Well_58-32_processed_pason_log.csv')

In [ ]:
gdc19.get_drilling_path('Well_58-32_raw_pason_log.csv')

In [ ]:
# TODO

## Injection data

In [ ]:
injections = []
for f in gdc19.list_filenames('injection'):
    injections.append(pd.read_csv(gdc19.get_injection_path(f)))

In [ ]:
injections[0].head()

In [ ]:
injections[1].head()

In [ ]:
# TODO

## Now Save out the Project

In [ ]:
# And now create an OMF project file for the surfaces
proj = omf.Project(
    name='FORGE Surfaces',
    description='All data for the 2019 FORGE Geothermal Student Competition '
)

In [ ]:
proj.elements = [
                 # Surfaces
                 land_surface, 
                 temp_225c, 
                 temp_175c, 
                 opal_mound_fault,
                 negro_mag_fault,
                 top_granitoid,
                 # Pointsets
                 boundary,
                 temperature,
                 # Well Data
                 # TODO
                 # Drilling Data
                 # TODO
                 # Injection Data
                 # TODO
                ]

In [ ]:
proj.validate()

In [ ]:
# Save the surface project file
omf.OMFWriter(proj, gdc19.get_omf_project_filename())